<a href="https://colab.research.google.com/github/Rubeennn/Dog_Cat_dataset/blob/main/Cat_Dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
! pip3 install -q kaggle

In [22]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/home/ruben_ghazaryan/.kaggle’: File exists


In [23]:
! cp kaggle.json ~/.kaggle/

In [24]:
! chmod 600 ~/.kaggle/kaggle.json

In [25]:
! kaggle datasets download -d unmoved/30k-cats-and-dogs-150x150-greyscale

100%|████████████████████████████████████████| 133M/133M [00:34<00:00, 4.01MB/s]
100%|████████████████████████████████████████| 133M/133M [00:34<00:00, 3.98MB/s]


In [6]:
from pathlib import Path

data_path = Path('data')

data_path.mkdir(exist_ok=True, parents=True)


In [27]:
import zipfile

with zipfile.ZipFile('30k-cats-and-dogs-150x150-greyscale.zip', 'r') as z:
    z.extractall(data_path)

In [7]:
cat_path = data_path / 'Animal_Images' / 'cats'
cat_path
dog_path = data_path / 'Animal_Images' / 'dogs'

### Importing the necessary packages

In [8]:
import numpy as np
from PIL import Image
import pandas as pd
import torch
from torch import nn
import torchvision
from torchvision.datasets import ImageFolder
from torchvision import transforms
import os

In [9]:
cat_image_names = os.listdir(cat_path)
first_cat_image_path = cat_path / cat_image_names[0]

In [10]:
img = Image.open(fp=first_cat_image_path)

In [11]:
np.asarray(img).shape

(150, 150)

In [12]:
path = Path(data_path / 'Animal_Images')

In [36]:
dataset = ImageFolder(
    root = path,
    transform=transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize(size=(124,124)),
    transforms.ToTensor()
    ])
)

In [37]:
type(dataset)

torchvision.datasets.folder.ImageFolder

In [38]:
torch.manual_seed(42)
from torch.utils.data import random_split, DataLoader

train_size = int(len(dataset) * 0.7)
test_size = int(len(dataset) * 0.15)
val_size = len(dataset) - train_size - test_size
train_dataset, test_dataset, val_dataset = random_split(dataset, 
                                                        [train_size, 
                                                         test_size,
                                                         val_size]
                                                       )


In [42]:
train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=64,
                              shuffle=True,
                              num_workers=os.cpu_count()
                             )
test_loader = DataLoader(dataset=test_dataset,
                        batch_size=64,
                        shuffle=False,
                        num_workers=os.cpu_count())
val_loader = DataLoader(dataset=val_dataset,
                       batch_size=64,
                       shuffle=False,
                       num_workers=os.cpu_count())

In [60]:
class DogCat(nn.Module):
    def __init__(self, input_shape, hidden_shape, output_shape=2):
        super().__init__()
        
        self.block1 = nn.Sequential(nn.BatchNorm2d(num_features=1),
                                   nn.Conv2d(in_channels=1,
                                             out_channels=3,
                                             kernel_size=(3,3),
                                             stride=1,
                                             padding=1
                                            ),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(num_features=3),
                                    nn.Conv2d(in_channels=3,
                                              out_channels=6,
                                              kernel_size=(3,3),
                                              stride=1,
                                              padding=1
                                    ),
                                    nn.MaxPool2d(kernel_size=(2,2)),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(num_features=6)
                                   )
    def forward(self, X):
        print(X.shape)
        X = self.block1(X)
        print(X.shape)
        return X

In [61]:
model = DogCat(input_shape=124,
              hidden_shape=10)

In [62]:
train_dataset[0]

(tensor([[[0.9216, 0.9294, 0.9294,  ..., 0.9765, 0.9725, 0.9686],
          [0.9569, 0.9569, 0.9569,  ..., 0.9765, 0.9686, 0.9686],
          [0.9765, 0.9725, 0.9686,  ..., 0.9725, 0.9647, 0.9608],
          ...,
          [0.6000, 0.6196, 0.6275,  ..., 0.6196, 0.6275, 0.6275],
          [0.6314, 0.6549, 0.6588,  ..., 0.6118, 0.6157, 0.6157],
          [0.7255, 0.7412, 0.7333,  ..., 0.6118, 0.6118, 0.6118]]]),
 1)

In [63]:
len(train_dataloader)

329

In [64]:
model.train()
for batch , (X,y) in enumerate(train_dataloader):
    if batch == 100:
        print(model(X))
    

torch.Size([64, 1, 124, 124])
torch.Size([64, 6, 62, 62])
tensor([[[[-5.7280e-01, -9.0583e-01, -7.8656e-01,  ..., -7.8260e-01,
           -7.9209e-01, -7.5512e-01],
          [-9.0583e-01, -9.0583e-01, -9.0583e-01,  ..., -4.7712e-01,
           -5.2876e-01, -7.5603e-01],
          [-9.0583e-01, -9.0583e-01, -9.0583e-01,  ..., -7.7343e-01,
           -9.0583e-01, -9.0583e-01],
          ...,
          [-9.0583e-01, -9.0583e-01, -9.0583e-01,  ..., -9.0583e-01,
           -9.0583e-01, -8.9270e-01],
          [-9.0583e-01, -9.0583e-01, -9.0583e-01,  ..., -4.6458e-01,
           -9.0583e-01, -9.0583e-01],
          [-9.0583e-01, -9.0583e-01, -9.0583e-01,  ...,  1.2495e-01,
           -8.9053e-01, -8.1762e-01]],

         [[-1.1109e+00, -1.1109e+00, -1.1109e+00,  ...,  2.0070e-01,
           -4.0976e-01, -3.6645e-01],
          [-1.1109e+00, -1.1109e+00, -1.1109e+00,  ..., -4.5350e-01,
            1.1384e-02,  6.9854e-02],
          [-1.1109e+00, -1.1109e+00, -1.1109e+00,  ..., -1.0637e-01,
